# 光学的内径計測のシミュレーションを行う

- 前回シミュレーションの反省点
    - スリット通過後の受光面の位置がバラバラであった(スリット通過後のレンズによる集光を無視していた)
    - 受光面に関して分解能的概念を設けることを忘れていた -> 結果として、大きさに関わらずすべて相似形の結果が得られてしまうガバガバ設計になった
    
## 以上の反省点を活かし,新しいシミュレーション手法を考える
[概要]</br>slit,lens,P.Tを1セットとして円管周りを回転させ,目的の光をある閾値を超えた場合のみキャッチ</br>

<img src="./simulation.png" width="500"/>

## 必要となる関数
- 円管内部の屈折の状態を表す関数 calc_refraction
    - 入力：光が入射する座標, 空気と部材の屈折率
    - 出力：円管の外に出るときの座標,(Y軸となす角度(radian)) ← いらないかも


- 受光できるかできないかを表す関数
    - 入力：外周の座標,角度
    - 出力：強度,角度,座標

 
## 検出フロー
1.円管内部の屈折の状態を表す関数を使って円管の端から端の範囲の状態を記録 

2.スリットの条件,エネルギーの閾値(半導体ギャップ)を設定し,-π/2<θ<π/2の範囲で回転させ条件を超えられる場合のみ記録
→正確には円管とスリットまでの距離も定義が必要


$Eg < hν = hc/λ$　を満たすかどうか

## その他必要になったら付け足す関数
- 入射角,屈折角を計算する関数 calc_angle
- 解の公式 solv_quadratic_equation

In [1]:
import numpy as np

#解の公式
def solv_quadratic_equation(a_q, b_q, c_q):
    # 2次方程式を解く  
    D = (b_q**2 - 4*a_q*c_q) ** (1/2)
    if D < 0:
        #接していない
        print("--要編集--")
        return 1,1
    elif D == 0:
        #接する
        print("接しています")
        return 0,0
    else:
        #2点で接する
        x_1 = (-b_q + D) / (2 * a_q)
        x_2 = (-b_q - D) / (2 * a_q)

        return x_1, x_2

# 円管内部の屈折の状態を表す関数
def calc_refraction(n1, n2, x1, y1, x2, y2, a_i, b_i, a_r, b_r):
    #屈折の様子を定義
    #1.入射した座標から屈折後の角度を求める -> スネルの法則
    if n1 < n2: #上側のみを考えている状態 →　下側は状態が変わってくるから更に条件分岐があったほうがよい y2の位置によって変わりそう
        if y2 > 0:#y2の位置によって変わる光の屈折の状態を定義、具体的には傾きのとり方が変わる
            #n1→n2 基本的にn1<n2として考える
            #屈折角+水平軸と円の接線の法線が成す角
            sita = calc_angle(n1, n2, x1, y1, x2, y2)
            #tan(-sita)で傾きと(x2,y2)で座標から直線の式がわかる
            #円の式と直線の式から解の公式
            #解の公式に代入する用の変数を定義
            #xが0より大きいか小さいかで傾きに-がつくか変わるtan(sita)が大きさのため
            c = tan(-sita) if x< 0 else c = tan(sita)
            d = y2 -c*x2
            #楕円の式 x^2/a^2 + y^2/b^2 = 1 直線の式 y = cx + d
            A = (a_i**2)*(c**2)
            B = 2*(a_i**2)*c*d
            C = (a_i**2)*(d**2) - (a_i**2)*(b_i**2)
            #解の公式で次の座標を求める
            x_q1, x_q2 = solv_quadratic_equation(A, B, C)
            #D<0の場合はもう一度 → 外周の円との交点
            if x_q1 == 1 and x_q2 == 1:
                A = (a_r**2)*(c**2)
                B = 2*(a_r**2)*c*d
                C = (a_r**2)*(d**2) - (a_r**2)*(b_r**2)
                #解の公式で次の座標を求める
                x_q1, x_q2 = solv_quadratic_equation(A, B, C)
                x_ans = x_q2
                if x_ans < 0:
                    y_ans = -np.sqrt((b_i**2)*(1-(x**2)/(a_i**2)))
                else:
                    y_ans = np.sqrt((b_i**2)*(1-(x**2)/(a_i**2)))
                return x_ans, y_ans                
            #上側の場合はx_q2の座標を取得 -> xが負か正かで傾きが変わる #傾きで条件分岐させたほうが良いかも
            if x2 < 0:
                x_ans = x_q2
                y_ans = np.sqrt((b_i**2)*(1-(x**2)/(a_i**2)))
                return x_ans, y_ans
            else: #x2>0
                x_ans = x_q1
                y_ans = np.sqrt((b_i**2)*(1-(x**2)/(a_i**2)))
                return x_ans, y_ans                
            
        else: #y2<0
            
        
        
    else:
        #n2→n1
        #全反射も考慮
        #注意n1,n2を逆に入れる
        #こっちは屈折角only
        sita = calc_angle(n2, n1, x1, y1, x2, y2)
        
    return x, y, sita_ans


# 入射角,屈折角を計算する関数 calc_angle
def calc_angle(n1, n2, x1, y1, x2, y2, a, b):
    #円の接線の法線の傾き
    a = y2/(b**2)
    b = x2/(a**2)
    #光線の傾き
    c = y2 - y1
    d = x2 - x1
    sin_i = np.tan(abs(a*d - b*c)/abs(a*c - b*d))
    #全反射とそうでない場合を分ける #法線と光線の位置関係を追加してループできるようにする。
    if n1 < n2:
        #普通に透過
        #屈折角
        sita_r = np.arctan((n1/n2)*sin_i)
        #円の接線の法線の傾き
        a = y2
        b = x2
        #水平軸と円の接線の法線が成す角
        c = 0
        d = 1
        #屈折角+水平軸と円の接線の法線が成す角
        return katamuki = np.arctan(np.tan(abs(a*d - b*c)/abs(a*c - b*d))) + sita_r    
    else:
        #全反射せずに透過する場合もある
        #全反射記述
        return sita_r = np.arcsin(sin_i)
    
    
#楕円の式 x^2/a^2 + y^2/b^2 = 1 はこの変数で定義してね 
#a1,b1を外周の円 ,a2,b2を内周の円とする

#外で回し続けたほうが良いかも
while True: #need to change x,yが外周により外のとき
    x, y = calc_refraction(n1, n2, x1, y1, x2, y2, a2, b2, a1, b1)
    
    #最後にY軸となす角度を調べれば良さそう
    calc_angle(n1, n2, x1, y1, x2, y2, a1, b1)
    #外に出られる入射角だったら,その座標とY軸となす角度を記録
    if :
        break
        
#スリットらへんの処理

SyntaxError: invalid syntax (<ipython-input-1-98c7a5b459e2>, line 24)

12/20
現状回転無理そう、一旦一方向からの楕円だけで考える
- 進捗
楕円上側の内部に透過する場合の座標までとることができた

- Todo
光線の傾きと法線の位置関係を確かめれば割と簡単に角度の状態を記述できそうかもしれん
座標をキャッチするとき光線の傾きの正負で考えていったほうがx<0とかで考えるより楽かも